In [22]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [23]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/test.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

# Do your crazy machine learning thing here :) ...

## 1. Exploratory data analysis

## 2. Preprocessing

## 3. Training

In [28]:
# Set hyperparameters

max_iters = 100
gamma = 0.01
lambda_ = 0.1

# Initialize weights
initial_w = np.random.rand(tX.shape[1],)

from implementations import *
# Train
weights_ls_GD, loss_ls_GD = least_squares_GD(y, tX, initial_w, max_iters, gamma)
# weights_ls_SGD, loss_ls_SGD = least_squares_SGD(y, tX, initial_w, max_iters, gamma)
weights_ls, loss_ls = least_squares(y, tX)
# weights_rr, loss_rr = ridge_regression(y, tX, lambda_)
# weights_lr, loss_lr = logistic_regression(y, tX, initial_w, max_iters, gamma)
# weights_reg_lr, loss_reg_lr = reg_logistic_regression(y, tX, lambda_, initial_w, max_iters, gamma)

/home/xavi_oliva/Documents/EPFL/Projects/epfl_ml/project1/scripts/implementations.py:12: RuntimeWarning: overflow encountered in matmul
  
/home/xavi_oliva/Documents/EPFL/Projects/epfl_ml/project1/scripts/implementations.py:11: RuntimeWarning: invalid value encountered in matmul
  grad = np.gradient(y, tx)


TypeError: cannot unpack non-iterable NoneType object

## 4. Evaluation

In [27]:
weights = weights_ls


(31,) (30,)


## Generate predictions and save output in csv format for submission:

In [ ]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '../data/submission.csv'
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)